# Import

In [ ]:
import re
#import regex as re
import json
import pickle
import random
import datetime
import numpy as np
from scipy import spatial
import tensorflow as tf
from gensim.models.keyedvectors import KeyedVectors

# Define functions

In [ ]:
def sum_caption(caption):
    caption_splitted = re.split("[^a-zåàâäæçéèêëîïôöœßùûüÿA-ZÅÀÂÄÆÇÉÈÊËÎÏÔÖŒÙÛÜŸ’\-]+",caption)
    caption_vector = np.array(300*[0])
    for c in caption_splitted:
#        caption_vector = np.sum([w2v[c] for c in caption_splitted])
        try:
            caption_vector = caption_vector + w2v[c]
        except KeyError:
            pass
    return caption_vector

# Load data

In [ ]:
train_path = "mscoco/captions_val2014.json"
with open(train_path, 'r') as train_file:
    train_dict = json.load(train_file)
    
    print("Number of images: {}".format(len(train_dict["images"])))
    for inst in train_dict["images"][:3]:
        print(inst)
    print()
    
    print("Number of annotations: {}".format(len(train_dict["annotations"])))
    for inst in train_dict["annotations"][:3]:
        print(inst)

### Re-structure

In [ ]:
# Maybe have dict keys as strings
own_dict = {}
for im in train_dict["images"]:
    own_dict[im["id"]] = {}
    own_dict[im["id"]]["url"] = im["flickr_url"]
for cap in train_dict["annotations"]:
    try:
        own_dict[cap["image_id"]]["captions"].append(cap["caption"])
    except KeyError:
        own_dict[cap["image_id"]]["captions"] = [cap["caption"]]

In [ ]:
len(own_dict)
for a in own_dict.items():
    print(a)
    break

# Build vocabulary

In [ ]:
#word_set = set()
#for inst in train_dict["annotations"][:3]:
#    word_set = word_set.union(re.split("[^a-zåàâäæçéèêëîïôöœßùûüÿA-ZÅÀÂÄÆÇÉÈÊËÎÏÔÖŒÙÛÜŸ’\-]+",inst["caption"].lower()))
#word_set.remove('')
#print(word_set)

from gensim import corpora
documents = []
for im in own_dict.values():
    documents.extend(im["captions"])
documents = [d.lower() for d in documents]

### Preview structure of document list

In [ ]:
print(len(documents))
print(documents[:10])

# Load word2vec

In [ ]:
w2v = KeyedVectors.load_word2vec_format('../../data/GoogleNews-vectors-negative300.bin.gz',binary=True)

### Try similarity

In [ ]:
query_image = random.choice(list(own_dict.keys()))
# Use all captions for query image
query_vector = sum_caption(own_dict[query_image]["captions"][1])
curr_best = {"im":0, "dist":float("inf"), "cap_nr":-1}
for im in own_dict.items():
    if im[0] != query_image:
        for j in range(len(im[1]["captions"])):
            temp_vector = sum_caption(own_dict[im[0]]["captions"][j])
            if spatial.distance.cosine(query_vector,temp_vector) < curr_best["dist"]:
                curr_best["dist"] = spatial.distance.cosine(query_vector,temp_vector)
                curr_best["im"] = im[0]
                curr_best["cap_nr"] = j
                
# Show images
import webbrowser
webbrowser.open_new(own_dict[query_image]["url"])
webbrowser.open_new_tab(own_dict[curr_best["im"]]["url"])
print(own_dict[query_image]["url"])
print(own_dict[query_image]["captions"][1])
print(own_dict[curr_best["im"]]["url"])
print(own_dict[curr_best["im"]]["captions"][curr_best["cap_nr"]])